# <font color='orange'>Catégorisez automatiquement des questions</font>

##  <font color='navy'>Sommaire

**Partie 1 : Construction du Bag-of-words**
- <a href="#C1">Récupération des données</a>
- <a href="#C5">Nettoyage des données texte</a>
- <a href="#C10">Vectorisation du texte</a>
- <a href="#C15">Exploration des données</a>

**Partie 2 : Modelisation non supervisée**
- T-SNE avec TF-IDF (représentation graphique)
- <a href="#C1">LDA</a>
- <a href="#C5">NMF</a>


# <font color='navy'>PARTIE 2 - Modélisation non supervisée</font>

#### <font color='orange'>**Import des librairies python.**</font>

In [4]:
import pandas as pd
import pickle
from scipy import sparse

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
# Chargement de la matrice BoW.
with open('bow_matrix_sparse.pkl', 'rb') as f:
    sparse_bow_matrix = pickle.load(f)

# Affichage de la forme.
print(sparse_bow_matrix.shape)

(4961, 1000)


In [8]:
# Chargement de la matrice BoW.
with open('bow_matrix_sparse.pkl', 'rb') as f:
    sparse_bow_matrix = pickle.load(f)

# Affichage de la forme
print(f"Shape de la matrice BoW : {sparse_bow_matrix.shape}")

# Chargement du vectorizer
with open('tfidf_vectorizer.pkl', 'rb') as f:
    bow_vectorizer = pickle.load(f)

print("Le vectorizer a été chargé avec succès.")

Shape de la matrice BoW : (4961, 1000)
Le vectorizer a été chargé avec succès.


In [10]:
# Chargement de la matrice TF-IDF
with open('tfidf_matrix_sparse.pkl', 'rb') as f:
    sparse_tfidf_matrix = pickle.load(f)

# Affichage de la forme
print(f"Shape de la matrice TF-IDF : {sparse_tfidf_matrix.shape}")

# Chargement du vectorizer
with open('tfidf_vectorizer.pkl', 'rb') as f:
    tfidf_vectorizer = pickle.load(f)

print("Le vectorizer a été chargé avec succès.")

Shape de la matrice TF-IDF : (4961, 1000)
Le vectorizer a été chargé avec succès.


## <a name="C1"><font color='navy'>Latent Dirichlet Allocation (LDA)</font></a>

La LDA est un modèle non supervisé statistique qui permet de représenter des documents comme des mélanges de plusieurs sujets, et chaque sujet est lui-même un mélange de mots.

**Objectif du LDA :**

L'objectif de LDA est de déterminer à partir des documents existants :
- Les sujets présents dans l'ensemble des documents.
- La distribution de ces sujets dans chaque document.
- La distribution des mots dans chaque sujet.

**Comment ça marche :**

LDA commence par une hypothèse aléatoire sur la répartition des mots dans les sujets et des sujets dans les documents.
Ensuite, il ajuste ces hypothèses en fonction des mots qui apparaissent dans les documents réels.
Au fil de nombreuses itérations, le modèle affine ses hypothèses pour mieux correspondre aux données réelles.


C'est un outil puissant pour découvrir les thèmes dominants dans des collections de textes, comme des articles de recherche, des journaux, ou des messages sur les réseaux sociaux.

Nous utiliserons cette première approche pour détecter les topics associés aux mots de notre variable.

**LDA avec BOW**

In [15]:
# Initialisation du modèle LDA.
n_topics = 10  # Nombre de sujets à extraire
lda_model = LatentDirichletAllocation(n_components=n_topics, random_state=42, max_iter=5)

# Ajustement du modèle sur la matrice BoW.
lda_output = lda_model.fit_transform(sparse_bow_matrix)

# Analyse des topics.
def display_topics(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

# Affiche les 5 mots les plus importants de chaque topics.
display_topics(lda_model, bow_vectorizer.get_feature_names_out(), 5)

Topic 0:
export test true use symbol
Topic 1:
using version android build error
Topic 2:
file line using error tcp
Topic 3:
value string return object class
Topic 4:
error quotquot err npm ltdiv
Topic 5:
let using app user data
Topic 6:
public new class null return
Topic 7:
request error server user api
Topic 8:
gt const component function lt
Topic 9:
int return ltlt type code


**LDA avec TF-IDF**

In [17]:
import numpy as np
from sklearn.decomposition import LatentDirichletAllocation

# Initialisation de LDA.
n_topics = 10  # Nombre de topics à extraire
lda_model = LatentDirichletAllocation(n_components=n_topics, random_state=42, max_iter=5)
lda_output = lda_model.fit_transform(sparse_tfidf_matrix)

# Seuil de probabilité pour filtrer les topics avec un score supérieur à 0.5.
threshold = 0.5

# Récupérer les mots caractéristiques pour chaque topic.
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()

def get_topic_words(model, feature_names, n_top_words=5):
    """
    Récupère les mots les plus représentatifs pour chaque topic.

    Paramètres:
    -----------
    model: modèle LDA
    feature_names: noms des mots (tokens)
    n_top_words: nombre de mots par topic à afficher

    Retourne:
    --------
    list
        Liste des mots pour chaque topic.
    """
    topic_words = []
    for topic_idx, topic in enumerate(model.components_):
        top_words = " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        topic_words.append(top_words)
    return topic_words

# Obtenir les mots caractéristiques pour chaque topic
topic_words = get_topic_words(lda_model, tfidf_feature_names)

# Fonction pour obtenir les topics avec une probabilité supérieure au seuil
def get_relevant_topics(document_topics, threshold=0.5):
    return [(topic_idx, prob) for topic_idx, prob in enumerate(document_topics) if prob > threshold]

# Limiter l'affichage aux 5 premiers documents
num_docs_to_display = 5

# Afficher les topics pertinents pour chaque document
for doc_idx, doc_topics in enumerate(lda_output[:num_docs_to_display]):
    relevant_topics = get_relevant_topics(doc_topics, threshold)
    if relevant_topics:  # Afficher uniquement s'il y a des topics pertinents
        print(f"Document {doc_idx}:")
        for topic_idx, prob in relevant_topics:
            print(f"    Topic {topic_idx} (Probabilité: {prob:.4f}) - Mots : {topic_words[topic_idx]}")

Document 0:
    Topic 9 (Probabilité: 0.8788) - Mots : int ltlt template clang compiler
Document 1:
    Topic 7 (Probabilité: 0.7775) - Mots : request user public api new
Document 2:
    Topic 3 (Probabilité: 0.7875) - Mots : gt return class component const
Document 3:
    Topic 0 (Probabilité: 0.8239) - Mots : value data column time function
Document 4:
    Topic 6 (Probabilité: 0.8522) - Mots : error file install project build


In [53]:
import numpy as np
from sklearn.decomposition import LatentDirichletAllocation

# Initialisation de LDA.
n_topics = 10  # Nombre de sujets à extraire
lda_model = LatentDirichletAllocation(n_components=n_topics, random_state=42, max_iter=5)
lda_output = lda_model.fit_transform(sparse_tfidf_matrix)

# Seuil de probabilité pour filtrer les topics avec le meilleur score.
threshold = 0.5

# Récupérer les mots caractéristiques pour chaque topic.
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()

def get_topic_words(model, feature_names, n_top_words=10):
    """
    Récupère les mots les plus représentatifs pour chaque topic.

    Paramètres:
    -----------
    model: modèle LDA
    feature_names: noms des mots (tokens)
    n_top_words: nombre de mots par topic à afficher

    Retourne:
    --------
    list
        Liste des mots pour chaque topic.
    """
    topic_words = []
    for topic_idx, topic in enumerate(model.components_):
        top_words = " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        topic_words.append(top_words)
    return topic_words

# Obtenir les mots caractéristiques pour chaque topic
topic_words = get_topic_words(lda_model, tfidf_feature_names)

# Fonction pour obtenir les topics avec une probabilité supérieure au seuil
def get_relevant_topics(document_topics, threshold=0.5):
    return [(topic_idx, prob) for topic_idx, prob in enumerate(document_topics) if prob > threshold]

# Afficher les topics pertinents pour chaque document
for doc_idx, doc_topics in enumerate(lda_output):
    relevant_topics = get_relevant_topics(doc_topics, threshold)
    if relevant_topics:
        print(f"Document {doc_idx}:")
        for topic_idx, prob in relevant_topics:
            print(f"    Topic {topic_idx} (Probabilité: {prob:.4f}) - Mots : {topic_words[topic_idx]}")

Document 0:
    Topic 8 (Probabilité: 0.8588) - Mots : error using file gt app code use public return work
Document 1:
    Topic 5 (Probabilité: 0.8704) - Mots : gt using file error image function use return test app
Document 2:
    Topic 2 (Probabilité: 0.8970) - Mots : error gt file using ltdependencygt data return get code new
Document 3:
    Topic 9 (Probabilité: 0.8696) - Mots : file gt error using use app user new function value
Document 4:
    Topic 1 (Probabilité: 0.8336) - Mots : gt file using error app user code return data string
Document 5:
    Topic 0 (Probabilité: 0.8557) - Mots : gt using file error use code get return app image
Document 6:
    Topic 7 (Probabilité: 0.8514) - Mots : using file error gt function use get return code value
Document 7:
    Topic 1 (Probabilité: 0.8322) - Mots : gt file using error app user code return data string
Document 8:
    Topic 9 (Probabilité: 0.8566) - Mots : file gt error using use app user new function value
Document 9:
    Topic 0

In [ ]:
STOP

In [19]:
# Initialisation du modèle LDA.
n_topics = 10  # Nombre de sujets à extraire
lda_model = LatentDirichletAllocation(n_components=n_topics, random_state=42, max_iter=5)

# Ajustement du modèle sur la matrice TF-IDF.
lda_output = lda_model.fit_transform(sparse_tfidf_matrix)

# Analyse des topics.
def display_topics(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

# Affiche les 5 mots les plus importants de chaque topics.
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
display_topics(lda_model, tfidf_feature_names, 5)

Topic 0:
value data column time function
Topic 1:
studio file window visual jupyter
Topic 2:
line file ltmodulegt androidlayoutheightwrapcontent instruction
Topic 3:
gt return class component const
Topic 4:
err audio webpack file table
Topic 5:
app android page using chrome
Topic 6:
error file install project build
Topic 7:
request user public api new
Topic 8:
height width color child ltdiv
Topic 9:
int ltlt template clang compiler


La méthode LDA ne donne pas de resultats pertinents.

In [ ]:
import numpy as np

# Distribution des sujets pour chaque document
doc_topic_dist = lda_model.transform(tfidf)
topic_distribution_per_doc = np.argmax(doc_topic_dist, axis=1)
print(topic_distribution_per_doc)

In [ ]:
# Initialisation du modèle LDA avec les meilleurs paramètres
lda_best = LatentDirichletAllocation(
    n_components=best_params['n_components'],
    learning_method=best_params['learning_method'],
    max_iter=best_params['max_iter'],
    doc_topic_prior=best_params['doc_topic_prior'],
    topic_word_prior=best_params['topic_word_prior'],
    random_state=1
)

# Entraînement du modèle
lda_best.fit(tfidf)

## <a name="C5"><font color='navy'>Negative Matrix Factorisation (NMF)</font></a>

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

# Configuration du TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, 
    min_df=2, 
    max_features=1000, 
    stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(cleaned_df['sentence_bow_lem'])
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()

no_topics = 20

# Exécution de NMF
nmf = NMF(n_components=no_topics, random_state=1, init='nndsvd')
nmf.fit(tfidf)

no_top_words = 10
display_topics(nmf, tfidf_feature_names, no_top_words)

In [ ]:
from sklearn.decomposition import NMF
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

# Configuration du TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=1000, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(cleaned_df['sentence_bow_lem'])
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()

# Définition du modèle NMF
nmf = NMF()

# Paramètres à tester pour GridSearch
param_grid = {
    'n_components': [5, 10, 15, 20],  # Nombre de sujets
    'alpha': [0.0, 0.1, 0.5, 1.0],  # Paramètre de régularisation
    'l1_ratio': [0.0, 0.5, 1.0],  # Ratio de régularisation L1
    'init': ['random', 'nndsvd'],  # Méthode d'initialisation
    'max_iter': [10, 20, 30],  # Nombre maximum d'itérations
}

# GridSearchCV pour NMF
nmf_grid_search = GridSearchCV(
    estimator=nmf,
    param_grid=param_grid,
    n_jobs=-1,  # Utilisation de tous les cœurs disponibles
    cv=3,  # Nombre de plis pour la validation croisée
    verbose=1  # Verbose pour afficher les progrès
)

nmf_grid_search.fit(tfidf)
print("Meilleurs paramètres pour NMF : ", nmf_grid_search.best_params_)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Bag-of-Words
vectorizer_bow = CountVectorizer()
X_bow = vectorizer_bow.fit_transform(cleaned_df['sentence_bow_lem'])

# TF-IDF
vectorizer_tfidf = TfidfVectorizer()
X_tfidf = vectorizer_tfidf.fit_transform(cleaned_df['sentence_bow_lem'])

Réduction de Dimension (Utilisation de LDA pour identifier les topics)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

# Appliquer LDA
lda = LatentDirichletAllocation(n_components=10, random_state=42)
lda.fit(X_tfidf)

# Affichage des topics
for idx, topic in enumerate(lda.components_):
    print(f"Topic #{idx}:")
    print([vectorizer_tfidf.get_feature_names_out()[i] for i in topic.argsort()[-10:]])

Visualisation des topics en 2D avec une réduction de dimension.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Réduction de dimension avec PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_tfidf.toarray())

plt.figure(figsize=(10, 8))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c='blue', alpha=0.5)
plt.title('Réduction de dimension avec PCA')
plt.xlabel('Composante Principale 1')
plt.ylabel('Composante Principale 2')
plt.show()

In [ ]:
STOP

In [ ]:
from sklearn.model_selection import train_test_split
X = 
y = df_tokenized['tags_cleaned']
# Division du jeu de données
X_train, X_test, y_train, y_test = train_test_split(X_bow_lem, y, test_size=0.2, random_state=42)

# Entraînement d'un modèle
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train, y_train)

# Prédiction sur le jeu de test
y_pred = model.predict(X_test)

# Évaluation du modèle
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

**Approche non supervisée : LDA + LDAviz**

In [ ]:
# Import des librairies.
import mlflow
import mlflow.sklearn
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
# Démarrage du run.
with mlflow.start_run():
    # Code de votre expérimentation ici
    pass

In [ ]:
# Suivi des paramètres et des métriques.
# Exemples de paramètres
n_estimators = 100
max_depth = 5

# Enregistrer les paramètres
mlflow.log_param("n_estimators", n_estimators)
mlflow.log_param("max_depth", max_depth)

# Entraîner un modèle
model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
model.fit(X_train, y_train)

# Prédiction et évaluation
predictions = model.predict(X_test)
acc = accuracy_score(y_test, predictions)

# Enregistrer les métriques
mlflow.log_metric("accuracy", acc)

In [ ]:
# Enregistrement du modèle entraîné.
mlflow.sklearn.log_model(model, "random_forest_model")

In [ ]:
# Fin du run.
mlflow.end_run()

In [ ]:
# Voici un exemple complet d'un script MLFlow.
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

# Charger des données
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)

# Démarrer un run MLFlow
with mlflow.start_run():
    n_estimators = 100
    max_depth = 5

    # Enregistrer les paramètres
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)

    # Entraîner le modèle
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    model.fit(X_train, y_train)

    # Prédire et évaluer
    predictions = model.predict(X_test)
    acc = accuracy_score(y_test, predictions)

    # Enregistrer les métriques
    mlflow.log_metric("accuracy", acc)

    # Enregistrer le modèle
    mlflow.sklearn.log_model(model, "random_forest_model")

    # Fin du run
    mlflow.end_run()

In [ ]:
# Voici un exemple complet d'un script MLFlow pour comparer plusieurs modèles.
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

# Charger les données
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)

# Entraînement du premier modèle : RandomForest
with mlflow.start_run(run_name="RandomForest"):
    # Enregistrer les paramètres
    n_estimators = 100
    max_depth = 5
    mlflow.log_param("model_type", "RandomForest")
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    
    # Entraîner le modèle
    rf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    rf.fit(X_train, y_train)
    
    # Prédire et évaluer
    predictions = rf.predict(X_test)
    acc = accuracy_score(y_test, predictions)
    
    # Enregistrer les métriques
    mlflow.log_metric("accuracy", acc)
    
    # Enregistrer le modèle
    mlflow.sklearn.log_model(rf, "random_forest_model")

# Entraînement du deuxième modèle : LogisticRegression
with mlflow.start_run(run_name="LogisticRegression"):
    # Enregistrer les paramètres
    mlflow.log_param("model_type", "LogisticRegression")
    
    # Entraîner le modèle
    lr = LogisticRegression(max_iter=200)
    lr.fit(X_train, y_train)
    
    # Prédire et évaluer
    predictions = lr.predict(X_test)
    acc = accuracy_score(y_test, predictions)
    
    # Enregistrer les métriques
    mlflow.log_metric("accuracy", acc)
    
    # Enregistrer le modèle
    mlflow.sklearn.log_model(lr, "logistic_regression_model")

**Projet 5.**

In [ ]:
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec
import tensorflow_hub as hub
import numpy as np

# Import des données nettoyées.
# data = load_data()

In [ ]:
# PREPARATION DES DONNEES.

# Séparation des données en ensembles d'entraînement, de validation et de test.
X_train, X_temp, y_train, y_temp = train_test_split(data['text'], data['tags'], test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Début du run MLFlow pour la préparation des données.
with mlflow.start_run(run_name="Data Preparation"):
    # 1. Bag-of-Words.
    vectorizer = CountVectorizer(max_features=10000)
    X_train_bow = vectorizer.fit_transform(X_train).toarray()
    X_val_bow = vectorizer.transform(X_val).toarray()
    X_test_bow = vectorizer.transform(X_test).toarray()
    
    mlflow.log_param("embedding_type", "Bag-of-Words")
    mlflow.log_param("vocab_size", len(vectorizer.vocabulary_))

    # 2. Word2Vec.
    word2vec_model = Word2Vec(sentences=X_train, vector_size=300, window=5, min_count=1, workers=4)
    # Ensemble des données d'entraînement.
    X_train_w2v = np.array([np.mean([word2vec_model.wv[word] for word in sentence if word in word2vec_model.wv]
                                    or [np.zeros(300)], axis=0) for sentence in X_train])
    # Ensemble des données de validation.
    X_val_w2v = np.array([np.mean([word2vec_model.wv[word] for word in sentence if word in word2vec_model.wv] or [np.zeros(300)], axis=0)
    for sentence in X_val])
    # Ensemble des données de test.
    X_test_w2v = np.array([np.mean([word2vec_model.wv[word] for word in sentence if word in word2vec_model.wv] or [np.zeros(300)], axis=0)
    for sentence in X_test])
    
    mlflow.log_param("embedding_type", "Word2Vec")
    mlflow.log_param("vector_size", 300)

    # 3. BERT or USE (via TensorFlow Hub)
    embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
    X_train_use = embed(X_train)
    X_val_use = embed(X_val)
    X_test_use = embed(X_test)
    
    mlflow.log_param("embedding_type", "Universal Sentence Encoder")

    # Optionnel: Enregistrer des artefacts de préparation, comme les objets Vectorizer
    mlflow.log_artifact("vectorizer.pkl")

In [ ]:
# MODELISATION.
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

# Démarrer un run pour chaque modèle
with mlflow.start_run(run_name="Logistic Regression with BOW"):
    mlflow.log_param("model_type", "Logistic Regression")
    mlflow.log_param("embedding", "Bag-of-Words")
    
    model = LogisticRegression(max_iter=200)
    model.fit(X_train_bow, y_train)
    
    # Evaluation sur validation
    val_predictions = model.predict(X_val_bow)
    acc = accuracy_score(y_val, val_predictions)
    
    mlflow.log_metric("validation_accuracy", acc)
    
    mlflow.sklearn.log_model(model, "logistic_regression_bow_model")

with mlflow.start_run(run_name="SVM with Word2Vec"):
    mlflow.log_param("model_type", "SVM")
    mlflow.log_param("embedding", "Word2Vec")
    
    model = SVC(kernel='linear')
    model.fit(X_train_w2v, y_train)
    
    val_predictions = model.predict(X_val_w2v)
    acc = accuracy_score(y_val, val_predictions)
    
    mlflow.log_metric("validation_accuracy", acc)
    
    mlflow.sklearn.log_model(model, "svm_word2vec_model")

# Et ainsi de suite pour d'autres modèles comme MLP avec USE, etc.


In [ ]:
# EVALUATION DES MODELES.
from sklearn.metrics import precision_score, recall_score, f1_score

# Démarrer un run pour l'évaluation du modèle
with mlflow.start_run(run_name="Evaluation Logistic Regression BOW"):
    val_predictions = model.predict(X_val_bow)
    
    precision = precision_score(y_val, val_predictions, average='weighted')
    recall = recall_score(y_val, val_predictions, average='weighted')
    f1 = f1_score(y_val, val_predictions, average='weighted')
    
    # Enregistrer les métriques
    mlflow.log_metric("validation_precision", precision)
    mlflow.log_metric("validation_recall", recall)
    mlflow.log_metric("validation_f1", f1)
    
    # Taux de couverture des tags (exemple simplifié)
    # cover_rate = custom_coverage_function(y_val, val_predictions)
    # mlflow.log_metric("coverage_rate", cover_rate)

    # Enregistrer le modèle
    mlflow.sklearn.log_model(model, "final_logistic_regression_bow_model")

In [ ]:
# VALIDATION CROISEE.
from sklearn.model_selection import cross_val_score

with mlflow.start_run(run_name="Cross Validation Logistic Regression BOW"):
    cv_scores = cross_val_score(model, X_train_bow, y_train, cv=5)
    
    for i, score in enumerate(cv_scores):
        mlflow.log_metric(f"cv_fold_{i+1}_score", score)
    
    mlflow.log_metric("cv_mean_score", np.mean(cv_scores))

In [ ]:
STOP

In [ ]:
# Notebook 2: Approche Non Supervisée - LDA avec Visualisation en 2D.

# Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
import mlflow.sklearn

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE


# Charger les données pré-traitées
# Remplacer par le chemin de ton fichier ou méthode de chargement
data = pd.read_csv('data/questions_preprocessed.csv')

# Vectorisation avec CountVectorizer
vectorizer = CountVectorizer(max_features=10000)
X = vectorizer.fit_transform(data['concat_title_body']).toarray()

# Début d'un run MLFlow
with mlflow.start_run(run_name="LDA Model"):

    # Application de LDA
    lda = LatentDirichletAllocation(n_components=10, random_state=42)
    lda.fit(X)
    
    # Sauvegarde du modèle LDA
    mlflow.log_param("n_components", 10)
    mlflow.sklearn.log_model(lda, "lda_model")

    # Obtenir les topics
    feature_names = vectorizer.get_feature_names_out()
    topics = lda.components_
    for idx, topic in enumerate(topics):
        print(f"Topic {idx}:")
        print([feature_names[i] for i in topic.argsort()[-10:]])
    
    # Transformation des documents en topics
    topic_distribution = lda.transform(X)
    
    # Visualisation des topics en 2D avec t-SNE
    tsne = TSNE(n_components=2, random_state=42)
    topic_2d = tsne.fit_transform(topic_distribution)
    
    # Création du DataFrame pour la visualisation
    topic_df = pd.DataFrame(topic_2d, columns=['Dim1', 'Dim2'])
    topic_df['Tag'] = data['Tags']

    # Visualisation
    plt.figure(figsize=(10, 8))
    sns.scatterplot(data=topic_df, x='Dim1', y='Dim2', hue='Tag', palette='tab10')
    plt.title('Visualisation des Topics LDA')
    plt.show()
    
    # Comparaison avec les vrais tags
    true_tags = data['Tags'].apply(lambda x: x.split(','))  # Convertir les tags en liste si nécessaire
    for idx, topic in enumerate(topics):
        print(f"Vérification des vrais tags pour Topic {idx}:")
        topic_tags = set([feature_names[i] for i in topic.argsort()[-10:]])
        matching_tags = true_tags.apply(lambda tags: topic_tags.intersection(set(tags))).tolist()
        print(f"Matching tags pour Topic {idx}: {matching_tags}")

Y a t-il un sens dans le regroupement des mots par topics ? 
Cette approche est-elle pertinente ?